In [1]:
!pip install langchain langchain-community langchain_openai --quiet

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 36.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.4/62.4 kB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 433.9/433.9 kB 16.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 23.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 2.3 MB/s eta 0:00:00


In [2]:
from langchain.prompts import ChatPromptTemplate
from langchain.schema.output_parser import StrOutputParser
from langchain.schema.runnable import RunnableBranch
from langchain_openai import ChatOpenAI

In [3]:
import os
from google.colab import userdata
os.environ["OPENAI_API_KEY"] = userdata.get('practice-key')
model = ChatOpenAI(model="gpt-4o-mini")

In [4]:
positive_feedback_template = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a helpful assistant"),
        ("human", "Generate a response addressing this negative feedback: {feedback}"),
    ]
)

In [5]:
negative_feedback_template = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a helpful assistant"),
        ("human", "Generate a response addressing this negative feedback: {feedback}"),
    ]
)

In [6]:
neutral_feedback_template = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a helpful assistant"),
        ("human", "Generate a response addressing this neutral feedback: {feedback}"),
    ]
)

In [8]:
escalate_feedback_template = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a helpful assistant"),
        ("human", "Generate a message to escalate this feedback to a human agent: {feedback}"),
    ]
)

In [7]:
classification_template = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a helpful assistant"),
        ("human", "Classify this feedback as positive, negative, or neutral: {feedback}"),
    ]
)

In [9]:
branches = RunnableBranch(
    (
        lambda x: "positive" in x,
        positive_feedback_template | model | StrOutputParser() # positive feedback chain
    ),
    (
        lambda x: "negative" in x,
        negative_feedback_template | model | StrOutputParser()
    ),
    (
        lambda x: "neutral" in x,
        neutral_feedback_template | model | StrOutputParser()
    ),
    escalate_feedback_template | model | StrOutputParser()
)

In [10]:
classification_chain = classification_template | model | StrOutputParser()

chain = classification_chain | branches

In [11]:
review = "Thie product is excellent. I really enjoyed it and found it very helpful."

result = chain.invoke({"feedback": review})

print(result)

Thank you for your feedback! I'm glad to hear that you had a positive experience. If there are any specific areas you'd like to highlight or suggestions you have for improvement, please feel free to share. Your insights are valuable in helping us continue to enhance our services.


In [12]:
review = "Thie product is excellent. But try to be careful while packaging. The package was damaged"

result = chain.invoke({"feedback": review})

print(result)

Thank you for your feedback! We’re glad to hear that you found the product to be excellent, as we strive to deliver high-quality items to our customers. However, we sincerely apologize for the issue with the damaged packaging. We understand that packaging is an important part of the overall experience, and we take this matter seriously. 

Your input helps us improve, and we will be addressing this concern with our packaging team to ensure better handling in the future. If you have any further suggestions or if there is anything else we can assist you with, please don't hesitate to reach out. We appreciate your support and hope to provide you with a flawless experience next time!


In [13]:
review = "The product quality was bad. The expiration date is nearly there. Also very bad packaging!"

result = chain.invoke({"feedback": review})

print(result)

Subject: Addressing Your Feedback

Dear [Feedback Provider's Name],

Thank you for taking the time to share your concerns with us. We genuinely value all feedback, as it helps us improve and better meet the needs of our customers.

I’m sorry to hear that your experience did not meet your expectations. We strive to provide the best possible service, and I understand how disappointing it can be when we fall short of that goal. 

Could you please provide more specific details about your experience? This information will help us address the issue more effectively and ensure that it doesn’t happen again in the future.

Thank you once again for your feedback. We are committed to making the necessary improvements, and your insights are invaluable in this process.

Best regards,

[Your Name]  
[Your Position]  
[Your Company]  
[Your Contact Information]  
